#### East ASO Bias Analysis

In [1]:
import numpy as np
from datetime import datetime
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp
import psycopg2
import warnings
warnings.filterwarnings('ignore')
print('Libraries imported')

C:\ProgramData\Anaconda3\envs\API\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\envs\API\lib\site-packages\numpy\.libs\libopenblas.el2c6ple4zyw3eceviv3oxxgrn2nrfm2.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\envs\API\lib\site-packages\numpy\.libs\libopenblas.fb5ae2tyxyh2ijrdkgdgq3xbklktf43h.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\envs\API\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Libraries imported


In [2]:
path1='D:/KIE Square Analytics Templates/PD'
disc=pd.read_excel(path1+'/'+'Sample_Discount.xlsx')

In [3]:
disc.shape

(7000, 8)

In [4]:
disc.drop(columns={'Unnamed: 0'}, axis=1, inplace=True)

In [5]:
disc.head()

,Unnamed: 0.1,Sales_Region,Dealer_ID,Month,Discount,Total_Discount,Price_Support
0,3384,EAST,7000607,10'22,23563.750000,23563.750000,0.00
1,13938,EAST,7011361,08'22,294141.625000,419579.065000,125437.44
2,21845,EAST,7024545,09'22,881.770833,881.770833,0.00
3,43336,EAST,7036186,07'22,7673.645833,14392.365833,6718.72
4,36171,EAST,7033203,08'22,20609.166667,44827.906667,24218.74


#### Discount

In [6]:
df=disc.copy()
df.head()

,Unnamed: 0.1,Sales_Region,Dealer_ID,Month,Discount,Total_Discount,Price_Support
0,3384,EAST,7000607,10'22,23563.750000,23563.750000,0.00
1,13938,EAST,7011361,08'22,294141.625000,419579.065000,125437.44
2,21845,EAST,7024545,09'22,881.770833,881.770833,0.00
3,43336,EAST,7036186,07'22,7673.645833,14392.365833,6718.72
4,36171,EAST,7033203,08'22,20609.166667,44827.906667,24218.74


In [7]:
df.columns

Index(['Unnamed: 0.1', 'Sales_Region', 'Dealer_ID', 'Month', 'Discount',
       'Total_Discount', 'Price_Support'],
      dtype='object')

In [8]:
df['Month'].unique()

array(["10'22", "08'22", "09'22", "07'22", "05'22", "02'23", "03'23",
       "12'22", "11'22", "01'23", "06'22", "04'22", "03'22", "02'22"],
      dtype=object)

In [9]:
df['Sales_Region'].unique()

array(['EAST'], dtype=object)

In [10]:
# df=df[df['Region']=='EAST'].copy()

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0.1    7000 non-null   int64  
 1   Sales_Region    7000 non-null   object 
 2   Dealer_ID       7000 non-null   int64  
 3   Month           7000 non-null   object 
 4   Discount        7000 non-null   float64
 5   Total_Discount  7000 non-null   float64
 6   Price_Support   7000 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 382.9+ KB


In [12]:
# df['District'] = df['District'].str.upper()

In [13]:
df[df.duplicated()]

,Unnamed: 0.1,Sales_Region,Dealer_ID,Month,Discount,Total_Discount,Price_Support


In [14]:
subset_columns = ['Sales_Region','Dealer_ID','Month']
duplicates = df.duplicated(subset=subset_columns, keep=False)
df[duplicates]

,Unnamed: 0.1,Sales_Region,Dealer_ID,Month,Discount,Total_Discount,Price_Support


#### PD WSP File and PJP File

In [15]:
df1,df2=pd.read_excel(path1+'/'+'Master_PD_WSP_Sample.xlsx'), pd.read_excel(path1+'/'+'PJP_ASO_Sample.xlsx')

In [16]:
east_pd=df1.copy()
pjp_aso=df2.copy()

##### East PD WSP

In [17]:
east_pd.columns

Index(['Unnamed: 0', 'state', 'sh_location', 'district', 'date', 'gross_price',
       'net_price', 'dal_wsp_manual', 'ut_wsp_manual', 'dal_wsp_qlik',
       'ut_wsp_qlik', 'computed_pd', 'recommended_pd', 'cdqd',
       'discount_factor', 'cdqddiscount_factor'],
      dtype='object')

In [18]:
sel_cols= ['state','district', 'date',
       'gross_price','recommended_pd','dal_wsp_manual','ut_wsp_manual', 'cdqd', 'discount_factor', 'cdqddiscount_factor']
east_pd1=east_pd[sel_cols].copy()
east_pd1.rename(columns={'state':'State','district':'District','date':'Date',
                        'gross_price':'Bill_Price','recommended_pd':'Total_PD',
                        'dal_wsp_manual':'Dist_WSP','ut_wsp_manual':'C-Dist_WSP',
                        'cdqd':'CD+QD','discount_factor':'DF','cdqddiscount_factor': 'CD+QD+DF'}, inplace=True)

In [19]:
east_pd1[east_pd1['Dist_WSP']=='NaN']

,State,District,Date,Bill_Price,Total_PD,Dist_WSP,C-Dist_WSP,CD+QD,DF,CD+QD+DF


In [20]:
east_pd1['Dist_WSP'].unique()

array(['345', '370-375', '315', '360-365', '310', '350', '365-370', '380',
       '385', '355', '285', '340', '365', '377.5', '330', '335-340',
       '360', '375', '335', '355-360', '325', '320', '320-325', '375-380',
       '391', '390', '358', '353', '374', '370', '376.25',
       '386.94444444444446', '322.5', '318', '350-355', '280', '340-345',
       '290', '395', '372', '312', '330-335', '305', '357', '300', '327',
       '380-385', '331.1111111111111', '362', '325-330', '348',
       '396.6666666666667', '388', '316', '378', '385-390', '365.625',
       '364.64285714285717', '345-350', '295', '383.3333333333333',
       '362.5', '336', '368', '363', '328', '303', '351',
       '374.2857142857143', '367', '352', '339.375', '354',
       '343.0769230769231', '315-320', '329', '317', '322', '332', '323',
       '307', '359', '364', '338', '360.8333333333333', '368.4375',
       '360.625', '391.07142857142856', '343', '313', '379', '377',
       '367.5', '373.2352941176471', '371.6

In [21]:
east_pd1= east_pd1[east_pd1['Dist_WSP']!='NaN'].copy()
east_pd1.loc[east_pd1['C-Dist_WSP'] == '370-75', 'C-Dist_WSP'] = '370-375'
east_pd1.drop_duplicates(inplace=True)

In [22]:
def convert_to_integer(value):
    if isinstance(value, str) and '-' in value:
        # If the value is a string and '-' is present, split the value and check if it has a valid format
        parts = value.split('-')
        if len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
            start, end = int(parts[0]), int(parts[1])
            result = float((start + end) / 2)
            return round(result,2)
    # If the value is either an integer or a string without '-', convert it directly into an integer
    try:
        result = float(value)
        return round(result, 2)
    except ValueError:
        return None

east_pd1['Dist_WSP'] = east_pd1['Dist_WSP'].apply(convert_to_integer)
east_pd1['C-Dist_WSP'] = east_pd1['C-Dist_WSP'].apply(convert_to_integer)

In [23]:
subset_columns = ['State','District', 'Date']
duplicates = east_pd1.duplicated(subset=subset_columns, keep=False)
east_pd1[duplicates]

,State,District,Date,Bill_Price,Total_PD,Dist_WSP,C-Dist_WSP,CD+QD,DF,CD+QD+DF


In [24]:
east_pd1.head()

,State,District,Date,Bill_Price,Total_PD,Dist_WSP,C-Dist_WSP,CD+QD,DF,CD+QD+DF
0,West Bengal,MURSHIDABAD,2022-11-14,385.0,0.0,345.0,345.0,0.0,10.0,10
1,Bihar,LAKHISARAI,2023-01-16,422.0,27.0,372.5,372.5,4.0,6.0,10
2,OD,PURI,2022-09-18,360.0,5.0,315.0,315.0,4.5,5.5,10
3,Bihar,BUXAR,2022-11-18,419.0,35.0,362.5,NaN,4.0,6.0,10
4,OD,DEOGARH,2022-11-12,345.0,15.0,315.0,310.0,4.5,5.5,10


In [25]:
east_pd1['State'].unique()

array(['West Bengal', 'Bihar', 'OD', 'Jharkhand'], dtype=object)

In [26]:
east_pd1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7000 entries, 0 to 6999
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   State       7000 non-null   object        
 1   District    7000 non-null   object        
 2   Date        7000 non-null   datetime64[ns]
 3   Bill_Price  7000 non-null   float64       
 4   Total_PD    7000 non-null   float64       
 5   Dist_WSP    6999 non-null   float64       
 6   C-Dist_WSP  5218 non-null   float64       
 7   CD+QD       7000 non-null   float64       
 8   DF          7000 non-null   float64       
 9   CD+QD+DF    7000 non-null   int64         
dtypes: datetime64[ns](1), float64(6), int64(1), object(2)
memory usage: 601.6+ KB


In [27]:
east_pd1[east_pd1['District']=='UTTAR DINAJPUR']['Total_PD'].unique()

array([ 0.    ,  5.    , 25.    , 20.    , 10.    , 30.    , 12.5209,
       15.    , 40.    , 11.1332, 35.    ])

##### South PD WSP

In [28]:
# south_pd.columns

In [29]:
# sel_cols= ['state','district', 'billing_date',
#        'gross_pricebag_two','pd','dalmia_wsp','ut_wsp',]
# south_pd1=south_pd[sel_cols].copy()
# south_pd1.rename(columns={'state':'State','district':'District','billing_date':'Date',
#                         'gross_pricebag_two':'Bill_Price','pd':'Total_PD',
#                         'dalmia_wsp':'Dist_WSP','ut_wsp':'C-Dist_WSP'}, inplace=True)

In [30]:
# south_pd1.info()

In [31]:
# south_pd1 = south_pd1[south_pd1['Dist_WSP'].notnull()].copy()

In [32]:
# south_pd1['Date'].unique()

In [33]:
# date_formats = ['%d.%m.%Y', '%Y-%m-%d %H:%M:%S']

# parsed_dates = []
# for date_str in south_pd1['Date']:
#     for date_format in date_formats:
#         try:
#             parsed_date = pd.to_datetime(date_str, format=date_format)
#             parsed_dates.append(parsed_date)
#             break
#         except ValueError:
#             continue
#     else:
#         parsed_dates.append(pd.NaT)
# south_pd1['Date'] = parsed_dates

In [34]:
# south_pd1['Date'].unique()

In [35]:
# south_pd1.head()

##### NE PD WSP

In [36]:
# ne_pd.columns

In [37]:
# sel_cols= ['state','district', 'billing_date',
#        'gross_price_bag_two','suggested_pd','dalmiawsp','star_wsp',]
# ne_pd1=ne_pd[sel_cols].copy()
# ne_pd1.rename(columns={'state':'State','district':'District','billing_date':'Date',
#                         'gross_price_bag_two':'Bill_Price','suggested_pd':'Total_PD',
#                         'dalmiawsp':'Dist_WSP','star_wsp':'C-Dist_WSP'}, inplace=True)

In [38]:
# ne_pd1['C-Dist_WSP'].unique()

In [39]:
# ne_pd1 = ne_pd1[ne_pd1['Dist_WSP'].notnull()].copy()

In [40]:
# ne_pd1.loc[ne_pd1['C-Dist_WSP'] == 0., 'C-Dist_WSP'] = None

In [41]:
# ne_pd1['Date'].info()

In [42]:
# ne_pd1.head()

##### Combine All PD WSP FILEs

In [43]:
# pd_wsp = pd.concat([east_pd1, south_pd1, ne_pd1], ignore_index=True)

In [44]:
# pd_wsp.info()

In [45]:
pd_wsp=east_pd1.copy()

In [46]:
pd_wsp['District'] = pd_wsp['District'].str.upper()
pd_wsp['Year-Month'] = pd_wsp['Date'].dt.strftime('%Y-%m')
# Convert the 'month' column to datetime type
pd_wsp['Year-Month'] = pd.to_datetime(pd_wsp['Year-Month'])
# Extract the month and year from the 'month' column
pd_wsp['Month'] = pd_wsp['Year-Month'].dt.strftime('%m\'%y')
# Drop the original 'month' column if needed
pd_wsp.drop(columns=['Year-Month'], inplace=True)

In [47]:
pd_wsp.head()

,State,District,Date,Bill_Price,Total_PD,Dist_WSP,C-Dist_WSP,CD+QD,DF,CD+QD+DF,Month
0,West Bengal,MURSHIDABAD,2022-11-14,385.0,0.0,345.0,345.0,0.0,10.0,10,11'22
1,Bihar,LAKHISARAI,2023-01-16,422.0,27.0,372.5,372.5,4.0,6.0,10,01'23
2,OD,PURI,2022-09-18,360.0,5.0,315.0,315.0,4.5,5.5,10,09'22
3,Bihar,BUXAR,2022-11-18,419.0,35.0,362.5,NaN,4.0,6.0,10,11'22
4,OD,DEOGARH,2022-11-12,345.0,15.0,315.0,310.0,4.5,5.5,10,11'22


In [48]:
# pd_wsp_1=pd_wsp.groupby(['State','District', 'Month'], as_index=False).apply(lambda group: pd.Series({
#     'M_Bill_Price': group['Bill_Price'].mean(), 
#     'M_Total_PD': group['Total_PD'].mean(),
#     'M_Dist_WSP': group['Dist_WSP'].mean(),
#     'M_C-Dist_WSP': group['C-Dist_WSP'].mean(),
# }))

In [49]:
# pd_wsp = pd.merge(pd_wsp,pd_wsp_1, how='inner',left_on=['State','District', 'Month'],right_on=['State','District', 'Month'])

In [50]:
pd_wsp[pd_wsp['District']=='UTTAR DINAJPUR']['Total_PD'].unique()

array([ 0.    ,  5.    , 25.    , 20.    , 10.    , 30.    , 12.5209,
       15.    , 40.    , 11.1332, 35.    ])

In [51]:
pd_wsp.head()

,State,District,Date,Bill_Price,Total_PD,Dist_WSP,C-Dist_WSP,CD+QD,DF,CD+QD+DF,Month
0,West Bengal,MURSHIDABAD,2022-11-14,385.0,0.0,345.0,345.0,0.0,10.0,10,11'22
1,Bihar,LAKHISARAI,2023-01-16,422.0,27.0,372.5,372.5,4.0,6.0,10,01'23
2,OD,PURI,2022-09-18,360.0,5.0,315.0,315.0,4.5,5.5,10,09'22
3,Bihar,BUXAR,2022-11-18,419.0,35.0,362.5,NaN,4.0,6.0,10,11'22
4,OD,DEOGARH,2022-11-12,345.0,15.0,315.0,310.0,4.5,5.5,10,11'22


In [52]:
subset_columns = ['State', 'District', 'Month', 'Date']
duplicates = pd_wsp.duplicated(subset=subset_columns, keep=False)
pd_wsp[duplicates]

,State,District,Date,Bill_Price,Total_PD,Dist_WSP,C-Dist_WSP,CD+QD,DF,CD+QD+DF,Month


In [53]:
# pd_wsp.to_excel('UPD.xlsx')

In [54]:
pd_wsp.head()

,State,District,Date,Bill_Price,Total_PD,Dist_WSP,C-Dist_WSP,CD+QD,DF,CD+QD+DF,Month
0,West Bengal,MURSHIDABAD,2022-11-14,385.0,0.0,345.0,345.0,0.0,10.0,10,11'22
1,Bihar,LAKHISARAI,2023-01-16,422.0,27.0,372.5,372.5,4.0,6.0,10,01'23
2,OD,PURI,2022-09-18,360.0,5.0,315.0,315.0,4.5,5.5,10,09'22
3,Bihar,BUXAR,2022-11-18,419.0,35.0,362.5,NaN,4.0,6.0,10,11'22
4,OD,DEOGARH,2022-11-12,345.0,15.0,315.0,310.0,4.5,5.5,10,11'22


#### PJP ASO 

In [55]:
pjp_aso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Unnamed: 0   100000 non-null  int64         
 1   visitdate    100000 non-null  datetime64[ns]
 2   referenceid  100000 non-null  object        
 3   dapid        100000 non-null  int64         
 4   partycode    93173 non-null   float64       
 5   product      100000 non-null  object        
 6   company      99986 non-null   object        
 7   wsp          100000 non-null  int64         
 8   rsp          98360 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 6.9+ MB


In [56]:
pjp_aso.head()

,Unnamed: 0,visitdate,referenceid,dapid,partycode,product,company,wsp,rsp
0,2180816,2022-05-26,EMP0006784,1756,77049927.0,DDSP,DALMIA,360,365.0
1,1502180,2023-08-22,EMP1027058,4376,7027105.0,ULTRATECH SUPER PPC,ULTRATECH,325,335.0
2,1525439,2023-08-24,EMP0010253,108631,77078697.0,DDSP,DALMIA,365,375.0
3,1416270,2023-08-11,EMP0009927,11692,7001476.0,ULTRATECH PPC,ULTRATECH,310,320.0
4,1403886,2023-08-10,EMP1038970,12673,7001611.0,DCFT,DALMIA,355,370.0


In [57]:
pjp_aso=pjp_aso[(pjp_aso['product']=='DCFT') & (pjp_aso['company']=='DALMIA')].copy()
sel_cols = ['visitdate', 'referenceid', 'partycode', 'wsp']
pjp_aso = pjp_aso[sel_cols]
pjp_aso.rename(columns={'visitdate': 'Date', 'referenceid':'ASO_ID','partycode':'Dealer_ID',
                  'wsp': 'ASO_WSP'}, inplace=True)
pjp_aso1=pjp_aso[(pjp_aso['Dealer_ID']>=7000000) & (pjp_aso['Dealer_ID']<=7100000)].copy()
pjp_aso1['ASO_ID'] = pjp_aso1['ASO_ID'].apply(lambda x: x.replace(' ', ''))
pjp_aso1=pjp_aso1[pjp_aso1['ASO_ID']!='AG00015'].copy()
pjp_aso1['Dealer_ID'] = pjp_aso1['Dealer_ID'].astype(int)
pjp_aso1 = pjp_aso1.groupby(['ASO_ID', 'Dealer_ID', 'Date'],as_index=False)['ASO_WSP'].mean()
pjp_aso1['Year-Month'] = pjp_aso1['Date'].dt.strftime('%Y-%m')
# Convert the 'month' column to datetime type
pjp_aso1['Year-Month'] = pd.to_datetime(pjp_aso1['Year-Month'])
# Extract the month and year from the 'month' column
pjp_aso1['Month'] = pjp_aso1['Year-Month'].dt.strftime('%m\'%y')
# Drop the original 'month' column if needed
pjp_aso1.drop(columns=['Year-Month'], inplace=True)

In [58]:
pjp_aso1.head()

,ASO_ID,Dealer_ID,Date,ASO_WSP,Month
0,EMP0000013,7015945,2022-04-20,415.0,04'22
1,EMP0000013,7015945,2022-07-28,375.0,07'22
2,EMP0000013,7016493,2022-10-06,400.0,10'22
3,EMP0000013,7016726,2022-08-17,370.0,08'22
4,EMP0000013,7016957,2022-10-18,400.0,10'22


In [59]:
# pjp_aso1.to_excel('ASO_WSP.xlsx')

#### Sales File

In [60]:
df5=pd.read_excel(path1+'/'+'Sales_Sample.xlsx')
sales1=df5.copy()

In [61]:
sales1.head()

,Unnamed: 0,sales_region,sales_zone,sold_to_party_district_name,billing_date,sold_to_party_code,sales_type,bill_quantity,gross_bill_price
0,381578,East,BHS,GAYA,2022-10-13,7027215,Trade,25.0,179000
1,2014124,East,BHS,ROHTAS,2023-03-30,7014927,Trade,7.5,61650
2,859505,East,WBS,MEDINAPUR WEST,2022-08-07,7032165,Trade,6.0,46920
3,1062125,East,ODW,ANGUL,2022-06-21,7000510,Trade,10.0,78000
4,1554702,East,ODW,SUNDARGARH,2022-04-10,7000481,Trade,10.0,73000


In [62]:
# sales1[sales1['sold_to_party_code']==7000004]['billing_date'].unique()

In [63]:
sel_cols=['sales_region','sold_to_party_district_name',
       'billing_date', 'sold_to_party_code', 'bill_quantity']
mb = sales1[sel_cols].copy()

In [64]:
mb['Year-Month'] = mb['billing_date'].dt.strftime('%Y-%m')

In [65]:
mon_d = mb.groupby(['sales_region','sold_to_party_district_name', 'sold_to_party_code', 'Year-Month', 'billing_date'], as_index=False).apply(lambda group: pd.Series({
    'Bill_Qty_Daily': group['bill_quantity'].sum()
}))

In [66]:
mon_d.rename(columns={'sales_region': 'Sales_Region', 'sold_to_party_district_name':'District',
                     'sold_to_party_code': 'Dealer_ID', 'billing_date':'Date'}, inplace=True)
mon_d['District'] = mon_d['District'].str.upper()
mon_d['Sales_Region'] = mon_d['Sales_Region'].str.upper()
mon_d= mon_d[mon_d['Sales_Region']=='EAST'].copy()
# mon_d.to_excel('East_Daily_Bill_Qty.xlsx')

In [67]:
mon = mb.groupby(['sales_region','sold_to_party_district_name', 'sold_to_party_code', 'Year-Month'], as_index=False).apply(lambda group: pd.Series({
    'Bill_Qty_Monthly': group['bill_quantity'].sum()
}))

In [68]:
mon.rename(columns={'sales_region': 'Sales_Region', 'sold_to_party_district_name':'District',
                     'sold_to_party_code': 'Dealer_ID'}, inplace=True)

In [69]:
mon.head()

,Sales_Region,District,Dealer_ID,Year-Month,Bill_Qty_Monthly
0,East,24 PRAGANA NORTH,7001073,2022-04,52.00
1,East,24 PRAGANA NORTH,7001073,2022-05,32.00
2,East,24 PRAGANA NORTH,7001073,2022-06,70.00
3,East,24 PRAGANA NORTH,7001073,2022-07,50.25
4,East,24 PRAGANA NORTH,7001073,2022-08,36.00


In [70]:
mon['District'] = mon['District'].str.upper()

In [71]:
mon['Sales_Region'] = mon['Sales_Region'].str.upper()

In [72]:
mon['Sales_Region'].unique()

array(['EAST'], dtype=object)

In [73]:
mon['Sales_Region'] = mon['Sales_Region'].replace('NORTH EAST', 'NE')
mon['Sales_Region'] = mon['Sales_Region'].replace('NE', 'NORTHEAST')

In [74]:
# mon1.to_excel('Sales_Monthly_Bill_Qty.xlsx')

In [75]:
subset_columns = ['Sales_Region','District','Dealer_ID','Year-Month']
duplicates = mon.duplicated(subset=subset_columns, keep=False)
mon[duplicates]

,Sales_Region,District,Dealer_ID,Year-Month,Bill_Qty_Monthly


In [76]:
mon['Year-Month'].unique()

array(['2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09',
       '2022-10', '2022-12', '2023-01', '2023-02', '2022-11', '2023-03'],
      dtype=object)

In [77]:
mon['Year-Month'] = pd.to_datetime(mon['Year-Month'])
mon['Month'] = mon['Year-Month'].dt.strftime('%m\'%y')
mon.drop(columns=['Year-Month'], inplace=True)

In [78]:
mon['Month'].unique()

array(["04'22", "05'22", "06'22", "07'22", "08'22", "09'22", "10'22",
       "12'22", "01'23", "02'23", "11'22", "03'23"], dtype=object)

#### Dealer Level

In [79]:
dle=pd.read_excel(path1+'/'+"Dealer_Levels.xlsx")

In [80]:
dle.head()

,Unnamed: 0,Region,Cust Code,Customer Name,DOA,MOA,Zone,Tier
0,0,EAST,7000001,KONARK CEMENT SUPPLY,2007-04-18,2007-04-01 00:00:00,ODC,No Tier
1,1,EAST,7000002,SWASTIK TRADERS,2007-04-20,2007-04-01 00:00:00,ODW,No Tier
2,2,EAST,7000003,JAISWAL CEMENT HOUSE,2007-04-20,2007-04-01 00:00:00,ODW,No Tier
3,3,EAST,7000004,SAHOO FERTILISERS,2007-05-10,2007-05-01 00:00:00,ODC,Bronze
4,4,EAST,7000005,MANORANJAN PANDA,2007-05-15,2007-05-01 00:00:00,ODC,No Tier


In [81]:
dle.columns

Index(['Unnamed: 0', 'Region', 'Cust Code', 'Customer Name', 'DOA', 'MOA',
       'Zone', 'Tier'],
      dtype='object')

In [82]:
sel_cols=['Cust Code','Tier']
dle1=dle[sel_cols].copy()
dle1.rename(columns={'Cust Code':'Dealer_ID', 'Tier': 'Dealer_Level'}, inplace=True)

In [83]:
# sel_cols = ['sold_to_party_district_name','sold_to_party_code','billing_date','bill_quantity','gross_bill_price']
# dle=dle[sel_cols].copy()
# dle.rename(columns={'sold_to_party_district_name': 'District','sold_to_party_code':'Dealer_ID'}, inplace=True)

In [84]:
# dle1 = dle.groupby(['Dealer_ID'], as_index=False).apply(lambda group: pd.Series({
# 'Volume': group['bill_quantity'].sum(), 'Sales': group['gross_bill_price'].sum(),
# }))

In [85]:
# dle1['Dealer_Level_Volume']='None'
# dle1['Dealer_Level_Sales']='None'
# dle1.loc[(dle1['Sales'] >= 10000000) & (dle1['Sales'] < 50000000), 'Dealer_Level_Sales'] = 'Medium'
# dle1.loc[(dle1['Sales'] >= 50000000), 'Dealer_Level_Sales'] = 'Big'
# dle1.loc[(dle1['Sales'] < 10000000), 'Dealer_Level_Sales'] = 'Small'
# dle1.loc[(dle1['Volume'] >= 1500) & (dle1['Volume'] < 7000), 'Dealer_Level_Volume'] = 'Medium'
# dle1.loc[(dle1['Volume'] >= 7000), 'Dealer_Level_Volume'] = 'Big'
# dle1.loc[(dle1['Volume'] < 1500), 'Dealer_Level_Volume'] = 'Small'

In [86]:
subset_columns = ['Dealer_ID']
duplicates = dle1.duplicated(subset=subset_columns, keep=False)
dle1[duplicates]

,Dealer_ID,Dealer_Level
34480,7037466,No Tier
34548,7037466,No Tier


In [87]:
dle1.head()

,Dealer_ID,Dealer_Level
0,7000001,No Tier
1,7000002,No Tier
2,7000003,No Tier
3,7000004,Bronze
4,7000005,No Tier


In [88]:
dle1['Dealer_Level'].unique()

array(['No Tier', 'Bronze', 'Mercury', 'Silver', 'Gold', 'Platinum',
       'Titanium'], dtype=object)

In [89]:
dle1.drop_duplicates(inplace=True)

In [90]:
mon.head()

,Sales_Region,District,Dealer_ID,Bill_Qty_Monthly,Month
0,EAST,24 PRAGANA NORTH,7001073,52.00,04'22
1,EAST,24 PRAGANA NORTH,7001073,32.00,05'22
2,EAST,24 PRAGANA NORTH,7001073,70.00,06'22
3,EAST,24 PRAGANA NORTH,7001073,50.25,07'22
4,EAST,24 PRAGANA NORTH,7001073,36.00,08'22


In [91]:
mon = pd.merge(mon,dle1, how='inner',left_on=['Dealer_ID'],right_on=['Dealer_ID'])

In [92]:
mon.head()

,Sales_Region,District,Dealer_ID,Bill_Qty_Monthly,Month,Dealer_Level
0,EAST,24 PRAGANA NORTH,7001073,52.00,04'22,Titanium
1,EAST,24 PRAGANA NORTH,7001073,32.00,05'22,Titanium
2,EAST,24 PRAGANA NORTH,7001073,70.00,06'22,Titanium
3,EAST,24 PRAGANA NORTH,7001073,50.25,07'22,Titanium
4,EAST,24 PRAGANA NORTH,7001073,36.00,08'22,Titanium


In [93]:
mon.columns

Index(['Sales_Region', 'District', 'Dealer_ID', 'Bill_Qty_Monthly', 'Month',
       'Dealer_Level'],
      dtype='object')

In [94]:
mon1 = mon.groupby(['Sales_Region', 'District','Month','Dealer_Level'], as_index=False).apply(lambda group: pd.Series({
    'Total_Bill_Qty_Monthly': group['Bill_Qty_Monthly'].sum()
}))

In [95]:
mon1.head()

,Sales_Region,District,Month,Dealer_Level,Total_Bill_Qty_Monthly
0,EAST,24 PRAGANA NORTH,01'23,Bronze,39.65
1,EAST,24 PRAGANA NORTH,01'23,Gold,40.50
2,EAST,24 PRAGANA NORTH,01'23,Mercury,5.00
3,EAST,24 PRAGANA NORTH,01'23,No Tier,7.00
4,EAST,24 PRAGANA NORTH,01'23,Platinum,17.00


In [96]:
mon= pd.merge(mon,mon1, how='inner',left_on=['Sales_Region', 'District','Month','Dealer_Level'],right_on=['Sales_Region', 'District','Month','Dealer_Level'])

In [97]:
mon.head()

,Sales_Region,District,Dealer_ID,Bill_Qty_Monthly,Month,Dealer_Level,Total_Bill_Qty_Monthly
0,EAST,24 PRAGANA NORTH,7001073,52.00,04'22,Titanium,172.75
1,EAST,24 PRAGANA NORTH,7001113,34.75,04'22,Titanium,172.75
2,EAST,24 PRAGANA NORTH,7001909,86.00,04'22,Titanium,172.75
3,EAST,24 PRAGANA NORTH,7001073,32.00,05'22,Titanium,217.85
4,EAST,24 PRAGANA NORTH,7001113,72.00,05'22,Titanium,217.85


#### Merge Sales with Discount

In [98]:
mon.head()

,Sales_Region,District,Dealer_ID,Bill_Qty_Monthly,Month,Dealer_Level,Total_Bill_Qty_Monthly
0,EAST,24 PRAGANA NORTH,7001073,52.00,04'22,Titanium,172.75
1,EAST,24 PRAGANA NORTH,7001113,34.75,04'22,Titanium,172.75
2,EAST,24 PRAGANA NORTH,7001909,86.00,04'22,Titanium,172.75
3,EAST,24 PRAGANA NORTH,7001073,32.00,05'22,Titanium,217.85
4,EAST,24 PRAGANA NORTH,7001113,72.00,05'22,Titanium,217.85


In [99]:
disc.head()

,Unnamed: 0.1,Sales_Region,Dealer_ID,Month,Discount,Total_Discount,Price_Support
0,3384,EAST,7000607,10'22,23563.750000,23563.750000,0.00
1,13938,EAST,7011361,08'22,294141.625000,419579.065000,125437.44
2,21845,EAST,7024545,09'22,881.770833,881.770833,0.00
3,43336,EAST,7036186,07'22,7673.645833,14392.365833,6718.72
4,36171,EAST,7033203,08'22,20609.166667,44827.906667,24218.74


In [100]:
disc['Sales_Region'].unique()

array(['EAST'], dtype=object)

In [101]:
dis_m = pd.merge(mon,disc, how='inner',left_on=['Sales_Region','Dealer_ID','Month'],right_on=['Sales_Region','Dealer_ID','Month'])

In [102]:
dis_m.head()

,Sales_Region,District,Dealer_ID,Bill_Qty_Monthly,Month,Dealer_Level,Total_Bill_Qty_Monthly,Unnamed: 0.1,Discount,Total_Discount,Price_Support
0,EAST,24 PRAGANA NORTH,7001073,50.25,07'22,Titanium,229.75,5185,283752.173800,4.351388e+05,151386.6
1,EAST,24 PRAGANA NORTH,7001113,38.00,10'22,Titanium,151.85,5278,457510.833333,4.575108e+05,0.0
2,EAST,24 PRAGANA NORTH,7037681,53.00,12'22,Titanium,247.25,48167,347303.879167,5.838439e+05,236540.0
3,EAST,24 PRAGANA NORTH,7001073,35.50,02'23,Titanium,269.00,5180,157584.383333,3.404584e+05,182874.0
4,EAST,24 PRAGANA NORTH,7001909,124.50,02'23,Titanium,269.00,6957,829819.385467,1.534209e+06,704390.0


In [103]:
dis_m['Discount/Bag']=(dis_m['Discount']/dis_m['Bill_Qty_Monthly'])/20
dis_m['Total_Discount/Bag']=(dis_m['Total_Discount']/dis_m['Bill_Qty_Monthly'])/20
dis_m['Price_Support/Bag']=(dis_m['Price_Support']/dis_m['Bill_Qty_Monthly'])/20

In [104]:
dis_m.head()

,Sales_Region,District,Dealer_ID,Bill_Qty_Monthly,Month,Dealer_Level,Total_Bill_Qty_Monthly,Unnamed: 0.1,Discount,Total_Discount,Price_Support,Discount/Bag,Total_Discount/Bag,Price_Support/Bag
0,EAST,24 PRAGANA NORTH,7001073,50.25,07'22,Titanium,229.75,5185,283752.173800,4.351388e+05,151386.6,282.340471,432.973904,150.633433
1,EAST,24 PRAGANA NORTH,7001113,38.00,10'22,Titanium,151.85,5278,457510.833333,4.575108e+05,0.0,601.987939,601.987939,0.000000
2,EAST,24 PRAGANA NORTH,7037681,53.00,12'22,Titanium,247.25,48167,347303.879167,5.838439e+05,236540.0,327.645169,550.796112,223.150943
3,EAST,24 PRAGANA NORTH,7001073,35.50,02'23,Titanium,269.00,5180,157584.383333,3.404584e+05,182874.0,221.949836,479.518850,257.569014
4,EAST,24 PRAGANA NORTH,7001909,124.50,02'23,Titanium,269.00,6957,829819.385467,1.534209e+06,704390.0,333.260797,616.148348,282.887550


In [105]:
dis_m[dis_m.duplicated()]

,Sales_Region,District,Dealer_ID,Bill_Qty_Monthly,Month,Dealer_Level,Total_Bill_Qty_Monthly,Unnamed: 0.1,Discount,Total_Discount,Price_Support,Discount/Bag,Total_Discount/Bag,Price_Support/Bag


In [106]:
subset_columns = ['Sales_Region', 'District', 'Dealer_ID','Month']
duplicates = dis_m.duplicated(subset=subset_columns, keep=False)
dis_m[duplicates]

,Sales_Region,District,Dealer_ID,Bill_Qty_Monthly,Month,Dealer_Level,Total_Bill_Qty_Monthly,Unnamed: 0.1,Discount,Total_Discount,Price_Support,Discount/Bag,Total_Discount/Bag,Price_Support/Bag


In [107]:
dis_m['District'] = dis_m['District'].str.upper()

In [108]:
dis_m.head()

,Sales_Region,District,Dealer_ID,Bill_Qty_Monthly,Month,Dealer_Level,Total_Bill_Qty_Monthly,Unnamed: 0.1,Discount,Total_Discount,Price_Support,Discount/Bag,Total_Discount/Bag,Price_Support/Bag
0,EAST,24 PRAGANA NORTH,7001073,50.25,07'22,Titanium,229.75,5185,283752.173800,4.351388e+05,151386.6,282.340471,432.973904,150.633433
1,EAST,24 PRAGANA NORTH,7001113,38.00,10'22,Titanium,151.85,5278,457510.833333,4.575108e+05,0.0,601.987939,601.987939,0.000000
2,EAST,24 PRAGANA NORTH,7037681,53.00,12'22,Titanium,247.25,48167,347303.879167,5.838439e+05,236540.0,327.645169,550.796112,223.150943
3,EAST,24 PRAGANA NORTH,7001073,35.50,02'23,Titanium,269.00,5180,157584.383333,3.404584e+05,182874.0,221.949836,479.518850,257.569014
4,EAST,24 PRAGANA NORTH,7001909,124.50,02'23,Titanium,269.00,6957,829819.385467,1.534209e+06,704390.0,333.260797,616.148348,282.887550


In [109]:
dis_m[(dis_m['District'] == 'BALASORE') & (dis_m['Month'] == "11'22")]['Total_Bill_Qty_Monthly'].unique()

array([86.5, 23.3, 98.5,  7.5])

In [110]:
pd_wsp.head()

,State,District,Date,Bill_Price,Total_PD,Dist_WSP,C-Dist_WSP,CD+QD,DF,CD+QD+DF,Month
0,West Bengal,MURSHIDABAD,2022-11-14,385.0,0.0,345.0,345.0,0.0,10.0,10,11'22
1,Bihar,LAKHISARAI,2023-01-16,422.0,27.0,372.5,372.5,4.0,6.0,10,01'23
2,OD,PURI,2022-09-18,360.0,5.0,315.0,315.0,4.5,5.5,10,09'22
3,Bihar,BUXAR,2022-11-18,419.0,35.0,362.5,NaN,4.0,6.0,10,11'22
4,OD,DEOGARH,2022-11-12,345.0,15.0,315.0,310.0,4.5,5.5,10,11'22


In [111]:
pjp_aso1.head()

,ASO_ID,Dealer_ID,Date,ASO_WSP,Month
0,EMP0000013,7015945,2022-04-20,415.0,04'22
1,EMP0000013,7015945,2022-07-28,375.0,07'22
2,EMP0000013,7016493,2022-10-06,400.0,10'22
3,EMP0000013,7016726,2022-08-17,370.0,08'22
4,EMP0000013,7016957,2022-10-18,400.0,10'22


In [112]:
dis_m_pj = pd.merge(dis_m,pjp_aso1, how='inner',left_on=['Month', 'Dealer_ID'],right_on=['Month', 'Dealer_ID'])

In [113]:
dis_m_pj.head()

,Sales_Region,District,Dealer_ID,Bill_Qty_Monthly,Month,Dealer_Level,Total_Bill_Qty_Monthly,Unnamed: 0.1,Discount,Total_Discount,Price_Support,Discount/Bag,Total_Discount/Bag,Price_Support/Bag,ASO_ID,Date,ASO_WSP
0,EAST,24 PRAGANA NORTH,7001073,50.25,07'22,Titanium,229.75,5185,283752.173800,435138.773800,151386.60,282.340471,432.973904,150.633433,EMP1028535,2022-07-13,320.0
1,EAST,KOLKATA,7001073,31.00,03'23,Titanium,685.50,5181,184783.277133,327912.277133,143129.00,298.037544,528.890770,230.853226,EMP1028535,2023-03-06,345.0
2,EAST,24 PRAGANA NORTH,7009101,19.00,12'22,Gold,141.00,12147,145256.606667,243779.606667,98523.00,382.254228,641.525281,259.271053,EMP1028535,2022-12-12,365.0
3,EAST,24 PRAGANA NORTH,7028347,2.50,07'22,Bronze,30.50,27267,12696.894400,31954.634400,19257.74,253.937888,639.092688,385.154800,EMP1030472,2022-07-25,315.0
4,EAST,24 PRAGANA NORTH,7029345,15.00,07'22,Silver,192.50,28893,65818.520833,99505.940833,33687.42,219.395069,331.686469,112.291400,EMP0010062,2022-07-07,320.0


In [114]:
subset_columns = ['Sales_Region', 'District', 'Dealer_ID','ASO_ID','Month','Date']
duplicates = dis_m_pj.duplicated(subset=subset_columns, keep=False)
dis_m_pj[duplicates]

,Sales_Region,District,Dealer_ID,Bill_Qty_Monthly,Month,Dealer_Level,Total_Bill_Qty_Monthly,Unnamed: 0.1,Discount,Total_Discount,Price_Support,Discount/Bag,Total_Discount/Bag,Price_Support/Bag,ASO_ID,Date,ASO_WSP


In [115]:
# dis_m_pj.drop_duplicates(inplace=True)

In [116]:
subset_columns = ['State','District', 'Month','Date']
duplicates = pd_wsp.duplicated(subset=subset_columns, keep=False)
pd_wsp[duplicates]

,State,District,Date,Bill_Price,Total_PD,Dist_WSP,C-Dist_WSP,CD+QD,DF,CD+QD+DF,Month


In [117]:
pd_wsp = pd_wsp.drop_duplicates(subset=subset_columns, keep='last')

In [118]:
dis_m_pj['Sales_Region'].unique()

array(['EAST'], dtype=object)

In [119]:
dis_m_pj['Sales_Region'].value_counts()

EAST    351
Name: Sales_Region, dtype: int64

In [120]:
pd_wsp.drop_duplicates(inplace=True)

In [121]:
fin = pd.merge(dis_m_pj,pd_wsp, how='inner',left_on=['District', 'Month', 'Date'],right_on=['District', 'Month', 'Date'])

In [122]:
fin.head()

,Sales_Region,District,Dealer_ID,Bill_Qty_Monthly,Month,Dealer_Level,Total_Bill_Qty_Monthly,Unnamed: 0.1,Discount,Total_Discount,...,Date,ASO_WSP,State,Bill_Price,Total_PD,Dist_WSP,C-Dist_WSP,CD+QD,DF,CD+QD+DF
0,EAST,KOLKATA,7001073,31.00,03'23,Titanium,685.50,5181,184783.277133,327912.277133,...,2023-03-06,345.0,West Bengal,385.0,15.0000,353.0,358.0,5.0,5.0,10
1,EAST,24 PRAGANA NORTH,7014916,10.00,01'23,Silver,110.40,17507,66305.208333,112322.208333,...,2023-01-17,380.0,West Bengal,400.0,25.0000,368.0,375.0,5.0,5.0,10
2,EAST,24 PRAGANA SOUTH,7001123,46.00,03'23,Platinum,408.15,5307,160099.881900,307603.881900,...,2023-03-22,360.0,West Bengal,380.0,5.0000,350.0,355.0,5.0,5.0,10
3,EAST,ANGUL,7015045,27.00,12'22,Gold,162.45,17644,157742.679400,188137.679400,...,2022-12-08,330.0,OD,360.0,20.0000,330.0,330.0,4.5,5.5,10
4,EAST,BALASORE,7028007,56.25,12'22,Titanium,445.30,26669,361101.067727,454888.067727,...,2022-12-22,325.0,OD,340.0,0.2094,320.0,325.0,4.5,5.5,10


In [123]:
subset_columns = ['Sales_Region','State', 'District', 'Dealer_ID','ASO_ID','Month','Date']
duplicates = fin.duplicated(subset=subset_columns, keep=False)
fin[duplicates]

,Sales_Region,District,Dealer_ID,Bill_Qty_Monthly,Month,Dealer_Level,Total_Bill_Qty_Monthly,Unnamed: 0.1,Discount,Total_Discount,...,Date,ASO_WSP,State,Bill_Price,Total_PD,Dist_WSP,C-Dist_WSP,CD+QD,DF,CD+QD+DF


In [124]:
fin['Sales_Region'].unique()

array(['EAST'], dtype=object)

In [125]:
fin.to_excel(path1+'/'+'ASO_Details_East_Fin_1.xlsx')